In [1]:
# UT-TOR-DATA-PT-01-2020-U-C Group Project 3
# Final project
# Import 1997 & 2000 electoral data
# (c) Boris Smirnov

In [2]:
# Depedences
import pandas as pd

In [3]:
# Constants
cand_txt = '1997-2000/cand.txt'
edsum_txt = '1997-2000/edsum.txt'

In [4]:
# Dataset with per candidate per riding results
cand_df = pd.read_csv(cand_txt, sep='\t', encoding='cp1252')

In [5]:
cand_df.columns

Index(['event_number', 'ed_code', 'ballot_sequence_number',
       'candidate_first_name', 'candidate_middle_name', 'candidate_last_name',
       'candidate_gender_code', 'candidate_occupation_english',
       'candidate_occupation_french', 'elected_indicator',
       'incumbent_indicator', 'candidate_vote', 'candidate_vote_percentage',
       'candidate_residence', 'candidate_province_english',
       'candidate_province_french', 'candidate_party_english_name',
       'candidate_party_french_name'],
      dtype='object')

In [6]:
cand_df['candidate_party_english_name'].unique()

array(['Progressive Conservative', 'N.D.P.', 'Liberal', 'No Affiliation',
       'Reform', 'Natural Law Party', 'Green Party', 'C.H.P. of Canada',
       'Independent', 'Marxist-Leninist', 'Bloc Québécois',
       'Canadian Action', 'Canadian Alliance', 'Marijuana Party',
       'Communist'], dtype=object)

In [7]:
# Transformation rules
cand_drop = [
    'ballot_sequence_number',
    'candidate_first_name', 'candidate_middle_name', 'candidate_last_name',
    'candidate_gender_code', 'candidate_occupation_english', 'candidate_occupation_french',
    'elected_indicator', 'candidate_vote_percentage',
    'candidate_residence', 'candidate_province_english', 'candidate_province_french',
    'candidate_party_french_name'
]

cand_rename = {
    'event_number': 'Year',
    'ed_code': 'FED Id',
    'incumbent_indicator' : 'Incumbent',
    'candidate_vote': 'Votes',
    'candidate_party_english_name': 'Candidate Pid'
}

event_rename = {
    3600: 1997,
    3700: 2000
}

party_rename = {
    'Progressive Conservative': 'CPC',
    'N.D.P.': 'NDP',
    'Liberal': 'LIB',
    'No Affiliation': 'OTH',
    'Reform': 'CPC',
    'Natural Law Party': 'OTH',
    'Green Party': 'GRN',
    'C.H.P. of Canada': 'OTH',
    'Independent': 'OTH',
    'Marxist-Leninist': 'OTH',
    'Bloc Québécois': 'BQ',
    'Canadian Action': 'OTH',
    'Canadian Alliance': 'CPC',
    'Marijuana Party': 'OTH',
    'Communist': 'OTH'
}

In [8]:
# Transformations
cand_df.drop(columns=cand_drop, inplace=True)
cand_df.rename(columns=cand_rename, inplace=True)
cand_df['Year'] = cand_df['Year'].map(event_rename)
cand_df['FED Id'] = cand_df['FED Id'].map(lambda ed_code: ed_code[2:4] + ed_code[5:]).astype('int64')
cand_df['Incumbent'] = cand_df['Incumbent'].map({'Y': 1, 'N': 0})
cand_df['Candidate Pid'] = cand_df['Candidate Pid'].map(party_rename)

In [9]:
cand_df

,Year,FED Id,Incumbent,Votes,Candidate Pid
0,1997,10001,0,10332,CPC
1,1997,10001,0,12359,NDP
2,1997,10001,1,12929,LIB
3,1997,10001,0,1054,OTH
4,1997,10002,0,13884,CPC
...,...,...,...,...,...
3475,2000,61002,0,2273,CPC
3476,2000,62001,0,1410,NDP
3477,2000,62001,0,349,GRN
3478,2000,62001,1,5327,LIB


In [10]:
# Dataset with elections results per riding
edsum_df = pd.read_csv(edsum_txt, sep='\t', encoding='cp1252')

In [11]:
edsum_df.columns

Index(['event_number', 'ed_code', 'ed_english_name', 'ed_french_name',
       'province_name_english', 'province_name_french',
       'ed_rejected_ballot_count', 'ed_valid_vote_count',
       'ed_electors_on_list_count', 'total_poll_count', 'advance_poll_count',
       'mobile_poll_count', 'svr_group1_ballot_count',
       'svr_group2_ballot_count', 'svr_total_ballot_count', 'population_cnt',
       'census_year', 'judicial_recount_indictator', 'majority_count',
       'majority_percentage', 'voter_participation_percentage',
       'elected_first_name', 'elected_middle_name', 'elected_last_name',
       'elected_party_english_name', 'elected_party_french_name',
       'event_english_name', 'event_french_name', 'returning_officer'],
      dtype='object')

In [12]:
# Transformation rules
edsum_rename = {
    'event_number': 'Year',
    'ed_code': 'FED Id',
    'ed_english_name': 'FED Name',
    'ed_rejected_ballot_count': 'Invalid Ballots',
    'ed_valid_vote_count': 'Valid Ballots',
    'ed_electors_on_list_count': 'Electors',
    'population_cnt': 'Population',
    'elected_party_english_name': 'Winning Pid',
}

edsum_drop = [
    'ed_french_name', 'province_name_english', 'province_name_french',
    'total_poll_count', 'advance_poll_count', 'mobile_poll_count',
    'svr_group1_ballot_count', 'svr_group2_ballot_count', 'svr_total_ballot_count',
    'census_year', 'judicial_recount_indictator', 'majority_count', 'majority_percentage',
    'voter_participation_percentage',
    'elected_first_name', 'elected_middle_name', 'elected_last_name',
    'elected_party_french_name', 'event_english_name', 'event_french_name',
    'returning_officer',
    'Invalid Ballots'
]

In [13]:
# Transforming initial dataset
edsum_df.rename(columns=edsum_rename, inplace=True)
edsum_df['Total Ballots'] = edsum_df['Invalid Ballots'] + edsum_df['Valid Ballots']
edsum_df.drop(columns=edsum_drop, inplace=True)
edsum_df['Year'] = edsum_df['Year'].map(event_rename)
edsum_df['FED Id'] = edsum_df['FED Id'].map(lambda ed_code: ed_code[2:4] + ed_code[5:]).astype('int64')
edsum_df['Winning Pid'] = edsum_df['Winning Pid'].map(party_rename)
edsum_df

,Year,FED Id,FED Name,Valid Ballots,Electors,Population,Winning Pid,Total Ballots
0,1997,10001,Bonavista--Trinity--Conception,36674,68065,94842,LIB,36886
1,1997,10002,Burin--St. George's,30383,55869,79263,CPC,30570
2,1997,10003,Gander--Grand Falls,25681,58598,82408,LIB,25836
3,1997,10004,Humber--St. Barbe--Baie Verte,30272,55341,79398,LIB,30401
4,1997,10005,Labrador,12212,20197,30379,LIB,12259
...,...,...,...,...,...,...,...,...
597,2000,59033,Kootenay--Boundary--Okanagan,41512,64366,92916,CPC,41750
598,2000,59034,West Vancouver--Sunshine Coast,53258,83706,116383,CPC,53413
599,2000,60001,Yukon,13219,20901,30766,LIB,13272
600,2000,61002,Western Arctic,12840,24716,24730,LIB,12912


In [14]:
# Moving incumbent info from 'per candidate' dataset to 'per riding' dataset
# The problem is that in 1997 and 2000 there were 2 conservative parties, that I have to unite into one 'virtual conservative party'
# (they all would merge into one in 2003)
# This dataset is reduced - if a riding had 2 conservative candidates, their results are summed up
# if any one of them was an incumbent, this property is saved for a united 'virtual' candidate
parties_df = cand_df.groupby(['Year', 'FED Id', 'Candidate Pid']).sum()
parties_df = parties_df.reset_index()
# Handling the situation where both conservative candidates were incumbents
parties_df.loc[parties_df['Incumbent'] == 2, 'Incumbent'] = 1

In [15]:
# Adding winning party Id
incumbents_df = pd.merge(
    parties_df[['Year', 'FED Id', 'Incumbent', 'Candidate Pid']],
    edsum_df[['Year', 'FED Id', 'Winning Pid']],
    how='left', on=['Year', 'FED Id']
)
# Filtering only winning candidates
incumbents_df = incumbents_df[incumbents_df['Candidate Pid'] == incumbents_df['Winning Pid']]
incumbents_df = incumbents_df.reset_index()
incumbents_df
# now we have Incumbent property for each riding. Adding this property to the ridings dataset
edsum_df['Incumbent'] = incumbents_df['Incumbent']

In [16]:
# Final step: saving per elections year results
for year in [1997, 2000]:
    candidates_df = cand_df[cand_df['Year'] == year]
    ridings_df = edsum_df[edsum_df['Year'] == year]

    party_grp = candidates_df.groupby(by=['FED Id', 'Candidate Pid'])
    votes_df = party_grp['Votes'].sum()
    votes_df = votes_df.unstack(level=-1, fill_value=0)

    merged_df = pd.merge(ridings_df, votes_df, left_on='FED Id', right_index=True)
    output_df = merged_df[[
        'Year',
        'FED Id',
        'FED Name',
        'Winning Pid',
        #'Incumbent', information isn't available in newer datasets. Incumbent candidates can only be determined by comparison with previous election results
        'Population',
        'Electors',
        'Total Ballots',
        'Valid Ballots',
        'LIB',
        'CPC',
        'NDP',
        'BQ',
        'GRN',
        'OTH'
    ]]
    output_df.to_csv(f"elections_{year}.csv", index=False)

In [22]:
# Checking ridings
ridings1997_df = pd.read_csv('elections_1997.csv')
ridings2000_df = pd.read_csv('elections_2000.csv')
comparison_df = pd.merge(
    ridings1997_df[['FED Id', 'FED Name']],
    ridings2000_df[['FED Id', 'FED Name']],
    how='left', on='FED Id', suffixes=('_1997', '_2000')
)
comparison_df[comparison_df['FED Name_1997'] != comparison_df['FED Name_2000']]

,FED Id,FED Name_1997,FED Name_2000
11,12001,Bras d'Or,Bras d'Or--Cape Breton
18,12008,Sackville--Eastern Shore,Sackville--Musquodoboit Valley--Eastern Shore
24,13003,Charlotte,New Brunswick Southwest
29,13008,Moncton,Moncton--Riverview--Dieppe
32,24001,Abitibi,Abitibi--Baie-James--Nunavik
35,24004,Argenteuil--Papineau,Argenteuil--Papineau--Mirabel
38,24007,Beauport--Montmorency--Orléans,Beauport--Montmorency--Côte-de-Beaupré--Île-d'...
46,24015,Charlesbourg,Charlesbourg--Jacques-Cartier
49,24018,Chicoutimi,Chicoutimi--Le Fjord
61,24030,Lac-Saint-Jean,Lac-Saint-Jean--Saguenay
